In [2]:
from collections import defaultdict
import ee
import pandas as pd
import geopandas as gpd
import geemap

C:\Users\theodore.wong\AppData\Local\Programs\Python\Python39\lib\site-packages\geopandas\_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\theodore.wong\AppData\Local\Temp\ipykernel_2292\953723026.py:4: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this

In [52]:
ee.Authenticate()

Enter verification code: 4/1Adeu5BWe2VmAK4ZjwhiH91inYrakSgqRqojiZ2wvJkrk9yTjLLpPrAvUqzI

Successfully saved authorization token.


In [3]:
ee.Initialize()

In [4]:
# Global warming potentials from IPCC AR5 Table 8.7. Using GWP-100.
# https://www.ipcc.ch/site/assets/uploads/2018/02/WG1AR5_Chapter08_FINAL.pdf
GWP = {
    'CO2': 1,
    'N2O': 265,
    'CH4': 28
}

In [5]:
YEARS = (2000, 2005, 2010, 2015, 2016, 2017, 2018, 2019, 2020)

In [6]:
ch4_ic = ee.ImageCollection('projects/wri-datalab/SCL-Cities/EDGAR-GHG/CH4')
co2_exclshort_ic = ee.ImageCollection('projects/wri-datalab/SCL-Cities/EDGAR-GHG/CO2exorgshort')
co2_short_ic = ee.ImageCollection('projects/wri-datalab/SCL-Cities/EDGAR-GHG/CO2orgshort')
n2o_ic = ee.ImageCollection('projects/wri-datalab/SCL-Cities/EDGAR-GHG/N2O')

In [9]:
urbext = {}
urbext_data = {}
urbext[2015] = ee.FeatureCollection('projects/wri-datalab/SCL-Cities/urbanextents__bycountry_2015')
urbext_data[2015] = geemap.ee_to_pandas(urbext[2015])

urbext[2020] = ee.FeatureCollection('projects/wri-datalab/SCL-Cities/urbanextents__bycountry_2020')
urbext_data[2020] = geemap.ee_to_pandas(urbext[2020])

In [10]:
scale = 100

In [12]:
worldpop = ee.ImageCollection("WorldPop/GP/100m/pop")
popcodes = worldpop.filter(ee.Filter.equals('year', 2020)).aggregate_array('country').getInfo()

In [ ]:
emi_res = defaultdict(list)
pop_res = defaultdict(list)

emi_total = {}
pop_total = {}

for year in [2015, 2020]:
    emi_total[year] = []
    pop_total[year] = []
    print()
    print(year)
    ch4 = ch4_ic.filter(ee.Filter.eq('year', year)).first()
    co2_exclshort = co2_exclshort_ic.filter(ee.Filter.eq('year', year)).first()
    co2_short = co2_short_ic.filter(ee.Filter.eq('year', year)).first()
    n2o = n2o_ic.filter(ee.Filter.eq('year', year)).first()
    co2e = (co2_exclshort.add(co2_short).add(n2o.multiply(GWP['N2O'])).add(ch4.multiply(GWP['CH4']))).multiply(31536) # avg tonnes per sq-m per year
    for i in range(len(urbext_data[year])):
        ua = urbext_data[year].iloc[i]
        ua_f = urbext[year].filter(ee.Filter.eq('city_ids', str(ua['city_ids'])))
        geom = ua_f.geometry()
        if ua['country'] in popcodes:
            localpop_img = worldpop.filter(ee.Filter.equals('year', year)).filter(ee.Filter.equals('country', ua['country'])).select('population').first()
            localemi = co2e.multiply(ee.Image.pixelArea()).reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e12).getInfo()['b1']
            localpop = localpop_img.reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e12).getInfo()['population']

            emi_res[(ua['country'], year)].append(localemi)
            pop_res[(ua['country'], year)].append(localpop)
            emi_total[year].append(localemi)
            pop_total[year].append(localpop)
    
        print(i, end=' ')



2015
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 

In [ ]:
out_dicts = []

out_dicts.append(
    {
        'country': 'global',
        'urban_ghg_emissions_tonnesCO2e_{0}'.format(year): sum(emi_total),
        'urban_population_{0}'.format(year): sum(pop_total)
    }
)

country_list = [i for i in list(set([i[0] for i in list(emi_res.keys())])) if (type(i)==str and len(i)>1)]
country_list.sort()
for country in country_list:
    out_dicts.append(
        {
            'country': country,
            'urban_ghg_emissions_tonnesCO2e_{0}'.format(year): sum(emi_res[(country, year)]),
            'urban_population_{0}'.format(year): sum(pop_res[(country, year)])
        }
    )
res_pd = pd.DataFrame(out_dicts)
res_pd.to_csv('CTY-4_GHGemissions.csv')